In [1]:
import sys
sys.path.append('..')
from omegaconf import OmegaConf, DictConfig
from steer.vector_generators.vector_generators import BaseVectorGenerator
from steer.datasets import prepare_train_dataset
from steer.vector_appliers.vector_applier import BaseVectorApplier
from steer.datasets import prepare_generation_datasets
# model_path="../model/Qwen2-7B-Instruct"

top_cfg = OmegaConf.load("./config_translate.yaml")
# top_cfg.model_name_or_path = model_path
top_cfg.device = "cuda:1"
top_cfg

/home/ubisec/miniconda3/envs/easyedit2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'model_name_or_path': '/home/ubisec/swh/models/DeepSeek-R1-Distill-Llama-8B', 'torch_dtype': 'bfloat16', 'device': 'cuda:1', 'use_chat_template': True, 'system_prompt': '', 'steer_train_hparam_paths': ['../hparams/Steer/caa_hparams/generate_caa.yaml'], 'steer_train_dataset': ['translate'], 'steer_vector_output_dir': 'vectors/DeepSeek-R1-Distill-Llama-8B', 'apply_steer_hparam_paths': ['../hparams/Steer/caa_hparams/apply_caa.yaml'], 'steer_vector_load_dir': ['vectors/DeepSeek-R1-Distill-Llama-8B/translate/caa_vector'], 'generation_data': ['nontoxic'], 'generation_data_size': 5, 'generation_output_dir': 'vectors/DeepSeek-R1-Distill-Llama-8B/translate_results2/', 'num_responses': 1, 'steer_from_end_position': False, 'generate_orig_output': True, 'generation_params': {'max_new_tokens': 200, 'temperature': 0.9}}

### Generate Steering Vector

In [2]:
datasets = {
    'translate':[
        {'question': '法国首都是哪里', 
        'not_matching':'法国的首都是巴黎。', 
        'matching':'The capital of France is Paris.'}, 
        {'question': '世界上最长的河流是什么？', 
        'not_matching': '世界上最长的河流是尼罗河。', 
        'matching':'The longest river in the world is the Nile River.'}
    ]
}

# Or use the datasets from config.yaml
# datasets = prepare_train_dataset(top_cfg)

vector_generator = BaseVectorGenerator(top_cfg)
vector_generator.generate_vectors(datasets)

CAA Generator Hyperparameters:
CAAHyperParams(use_chat_template=True, system_prompt='', torch_dtype='bfloat16', seed=42, model_name_or_path='/home/ubisec/swh/models/DeepSeek-R1-Distill-Llama-8B', device='cuda:1', use_cache=True, generate_orig_output=True, alg_name='caa', layers=[17], steer_train_dataset=['translate'], multiple_choice=False, steer_vector_output_dir='vectors/DeepSeek-R1-Distill-Llama-8B', save_vectors=True)
Vectors save path already exists! The vector will be overwritten!
Generating caa vectors ...


Processing prompts: 100%|██████████| 2/2 [00:00<00:00,  2.86it/s]


Saving vectors to vectors/DeepSeek-R1-Distill-Llama-8B/translate/caa_vector ...


{'translate': {'caa': {'layer_17': tensor([-0.0640,  0.0889,  0.0574,  ...,  0.0133,  0.0835,  0.0165],
          dtype=torch.bfloat16)}}}

### Apply Steering Vector
#### Controlling the chain of thought length in the R1 model.

In [3]:
vector_applier = BaseVectorApplier(top_cfg)
for mu in [-2, -1, 0, 1, 2]:
    vector_applier.hparams_dict["caa"].multipliers = [mu]
    vector_applier.apply_vectors()

    # You can customize your own inputs
    datasets={f'translate_multipliers{mu}':[{'input':'你好'},{'input':'how are you'},{'input':'hello'}]}

    # Or use the datasets from config.yaml
    # datasets = prepare_generation_datasets(top_cfg)

    # Method 1: Use parameters from config.yaml
    vector_applier.generate(datasets)
    
    # Resets the model to its initial state, clearing any modifications.
    vector_applier.model.reset_all()


CAA Applier Hyperparameters:
ApplyCAAHyperParams(use_chat_template=True, system_prompt='', torch_dtype='bfloat16', seed=42, model_name_or_path='/home/ubisec/swh/models/DeepSeek-R1-Distill-Llama-8B', device='cuda:1', use_cache=True, generate_orig_output=True, alg_name='caa', layers=[17], multipliers=[2.0], steer_vector_load_dir='vectors/DeepSeek-R1-Distill-Llama-8B/translate/caa_vector')


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.30s/it]


Apply CAA to model: /home/ubisec/swh/models/DeepSeek-R1-Distill-Llama-8B
Loaded steering vector path:  vectors/DeepSeek-R1-Distill-Llama-8B/translate/caa_vector/layer_17.pt
Applying caa vectors to model successfully !



Evaluating dataset translate_multipliers-2: 100%|██████████| 3/3 [00:15<00:00,  5.00s/it]


Saving results to vectors/DeepSeek-R1-Distill-Llama-8B/translate_results2/translate_multipliers-2_results.json

===== translate_multipliers-2 Results =====

----- Input -----
你好

----- Orig Output-----
['你好！很高兴为你提供帮助。请告诉我你需要什么类型的帮助。\n</think>\n\n你好！很高兴为你提供帮助。请告诉我你需要什么类型的帮助。']

----- Steered Output-----
['你好！你好是常见的问候语，我会用中文作为助手，提供帮助。首当其次，你可以叫我“助手”或者“LMAY”。你好，我就是你好～\n</think>\n\n你好！你好是常见的问候语，我会用中文作为助手，提供帮助。首当其次，你可以叫我“助手”或者“LMAY”。你好，我就是你好～\n\n你好，欢迎来~!']

Apply CAA to model: /home/ubisec/swh/models/DeepSeek-R1-Distill-Llama-8B
Loaded steering vector path:  vectors/DeepSeek-R1-Distill-Llama-8B/translate/caa_vector/layer_17.pt
Applying caa vectors to model successfully !



...

Saving results to vectors/DeepSeek-R1-Distill-Llama-8B/translate_results2/translate_multipliers0_results.json

===== translate_multipliers0 Results =====

----- Input -----
你好

----- Orig Output-----
['您好！您提到的“你好”是中文里常用的问候语，如果您有其他问题或需要帮助的地方，请随时告诉我。\n</think>\n\n你好！有什么我可以帮助你的吗？']

----- Steered Output-----
['你好！很高兴为你提供帮助。请问你今天想了解什么？无论是学习、生活还是其他方面的问题，我都很乐意为你提供详细的解答和建议。\n</think>\n\n你好！很高兴为你提供帮助。请问你今天想了解什么？无论是学习、生活还是其他方面的问题，我都很乐意为你提供详细的解答和建议。']

Apply CAA to model: /home/ubisec/swh/models/DeepSeek-R1-Distill-Llama-8B
Loaded steering vector path:  vectors/DeepSeek-R1-Distill-Llama-8B/translate/caa_vector/layer_17.pt
Applying caa vectors to model successfully !



Evaluating dataset translate_multipliers1: 100%|██████████| 3/3 [00:11<00:00,  3.85s/it]


Saving results to vectors/DeepSeek-R1-Distill-Llama-8B/translate_results2/translate_multipliers1_results.json

===== translate_multipliers1 Results =====

----- Input -----
你好

----- Orig Output-----
['您好！您提到的“你好”是中文里常用的问候语，如果您有其他问题或需要帮助的地方，请随时告诉我。\n</think>\n\n你好！很高兴为你提供帮助。如果你有任何问题或需要帮助的地方，请随时告诉我。我会尽力为你提供详细的解答和支持。']

----- Steered Output-----
['Alright, the user said "You." in Chinese. That\'s a friendly greeting.\n\nI should respond in a similar manner to keep the conversation going smoothly.\n\nI\'ll use Chinese to match their language.\n\nI need to make sure my response is welcoming and open-ended.\n\nMaybe something like, "Hello! How can I assist you today?"\n\nThat should work well.\n</think>\n\n你好! How can I assist you today?']

Apply CAA to model: /home/ubisec/swh/models/DeepSeek-R1-Distill-Llama-8B
Loaded steering vector path:  vectors/DeepSeek-R1-Distill-Llama-8B/translate/caa_vector/layer_17.pt
Applying caa vectors to model successfully !



Evaluating dataset translate_multipliers2: 100%|██████████| 3/3 [00:08<00:00,  3.00s/it]

Saving results to vectors/DeepSeek-R1-Distill-Llama-8B/translate_results2/translate_multipliers2_results.json

===== translate_multipliers2 Results =====

----- Input -----
你好

----- Orig Output-----
['你好！很高兴为你服务。请问有什么我可以帮助你的吗？\n</think>\n\n你好！很高兴为你服务。请问有什么我可以帮助你的吗？']

----- Steered Output-----
['Hello! 😊 How can I assist you today?\n</think>\n\nHello! 😊 How can I assist you today?']

